In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# Part 1 Data Exploration
# Based on 
# https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python/notebook

In [ ]:
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')

In [ ]:
# OverallQual, YearBuilt, TotalBsmtSF, GrLivArea might be important features
# Analyzing our target, SalePrice

In [ ]:
df_train['SalePrice'].describe()

In [ ]:
# histogram of SalePrice
sns.distplot(df_train['SalePrice'])

In [ ]:
# print out the skewness and kurtosis
print('Skewness: {}'.format(df_train['SalePrice'].skew()))
print('Kurtosis: {}'.format(df_train['SalePrice'].kurt()))

In [ ]:
# SalePrice's relationship with numerical variables

In [ ]:
data = pd.concat([df_train['SalePrice'], df_train['GrLivArea']], axis=1)
data.plot.scatter(x='GrLivArea', y='SalePrice', ylim=(0, 800000))

In [ ]:
data = pd.concat([df_train['SalePrice'], df_train['TotalBsmtSF']], axis=1)
data.plot.scatter(x='TotalBsmtSF', y='SalePrice', ylim=(0, 800000))

In [ ]:
# SalePrice's relationship with categorical features

In [ ]:
data = pd.concat([df_train['SalePrice'], df_train['OverallQual']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='OverallQual', y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);

In [ ]:
data = pd.concat([df_train['SalePrice'], df_train['YearBuilt']], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x='YearBuilt', y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);

In [ ]:
# objective analysis of the features

In [ ]:
# correlation matrix (heatmap style)
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
# SalePrice correlation matrix
cols = corrmat.nlargest(10, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
# scatter plots between 'SalePrice' and correlated variables
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols], size = 2.5)
plt.show();

In [ ]:
# Deal with missing data

In [ ]:
# missing data
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
# dealing with missing data
# delete the features that has more than 15% missing values
# GarageX variables are related to GarageCars, delete them, (same for Bsmt)
# MasVnrX are not essential and strongly correlated with 'YearBuilt' and 'OverallQual' (which are already been considered), delete them
# 1 missing for Electrical, delete the observation and keep the variable

df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train.isnull().sum().max()

In [ ]:
# Deal with outliers

In [ ]:
#Univariate Analysis

In [ ]:
# Standardizing the data
saleprice_scaled = StandardScaler().fit_transform(df_train['SalePrice'][:,np.newaxis]);
low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
high_range= saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]
print('outer range (low) of the distribution:')
print(low_range)
print('\nouter range (high) of the distribution:')
print(high_range)

In [ ]:
# Bivariate Analysis

In [ ]:
#bivariate analysis saleprice/grlivarea
data = pd.concat([df_train['SalePrice'], df_train['GrLivArea']], axis=1)
data.plot.scatter(x='GrLivArea', y='SalePrice', ylim=(0,800000));

In [ ]:
# two points with unusually large GrLivArea doesn't follow the general trend (may be agricultural area), delete
# two points with unusally high SalePrice are following the trend, should be special cases, keep
df_train.sort_values(by='GrLivArea', ascending=False)[:2]
df_train = df_train.drop(df_train[df_train.Id==1299].index)
df_train = df_train.drop(df_train[df_train['Id'] == 524].index)

In [ ]:
#bivariate analysis saleprice/grlivarea
data = pd.concat([df_train['SalePrice'], df_train['TotalBsmtSF']], axis=1)
data.plot.scatter(x='TotalBsmtSF', y='SalePrice', ylim=(0,800000));

In [ ]:
# Getting hard core
# (testing for the assumptions underlying the statistical bases for multivariate analysis)

In [ ]:
# Search for normality

In [ ]:
# histogram and normal probability plot for SalePrice
sns.distplot(df_train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

In [ ]:
# positive skewness -> log transformation
df_train['SalePrice'] = np.log(df_train['SalePrice'])
sns.distplot(df_train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

In [ ]:
# histogram and normal probability plot for GrLivArea
sns.distplot(df_train['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

In [ ]:
# positive skewness -> log transformation
df_train['GrLivArea'] = np.log(df_train['GrLivArea'])
sns.distplot(df_train['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

In [ ]:
#histogram and normal probability plot for TotalBsmtSF
sns.distplot(df_train['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['TotalBsmtSF'], plot=plt)

In [ ]:
# A significant number of observations with value zero (houses without basement)
# the value zero doesn't allow us to do log transformations
# create a variable that can get the effect of having or not having basement (binary variable) and do a log transformation to all the non-zero observations, ignoring those with value zero
# this way we can transform data, without losing the effect of having or not basement

df_train['HasBsmt'] = pd.Series(len(df_train['TotalBsmtSF']), index=df_train.index)
df_train['HasBsmt'] = 0
df_train.loc[df_train.TotalBsmtSF>0, 'HasBsmt'] = 1
df_train.loc[df_train.HasBsmt==1, 'TotalBsmtSF'] = np.log(df_train.loc[df_train.HasBsmt==1, 'TotalBsmtSF'])

In [ ]:
#histogram and normal probability plot
sns.distplot(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)

In [ ]:
# Search for homoscedasticity

In [ ]:
#scatter plot
plt.scatter(df_train['GrLivArea'], df_train['SalePrice']);
# log transformation made the original cone shape disappear

In [ ]:
#scatter plot
plt.scatter(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], df_train[df_train['TotalBsmtSF']>0]['SalePrice']);

In [ ]:
# Deal with dummy variables

In [ ]:
#convert categorical variable into dummy
df_train = pd.get_dummies(df_train)
df_train